In [155]:
import tensorflow as tf
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm

In [156]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-10000], y_train_full[:-10000]
X_valid, y_valid = X_train_full[-10000:], y_train_full[-10000:]
X_train, X_valid, X_test = X_train/255., X_valid/255., X_test/255.
X_train = X_train.reshape([-1, 28, 28, 1])
X_valid = X_valid.reshape([-1, 28, 28, 1])
X_test = X_test.reshape([-1, 28, 28, 1])

In [157]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
X_val = torch.tensor(X_valid, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.int64)
y_valid = torch.tensor(y_valid, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)


In [158]:
from torch import nn
from torch.nn import functional as F

In [159]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cpu


In [160]:
X_train = X_train.to(device)
X_test = X_test.to(device)
X_val = X_val.to(device)

y_train = y_train.to(device)
y_test = y_test.to(device)
y_valid = y_valid.to(device)

In [161]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        self.sequent = nn.Sequential(
            nn.Linear(28*28, 60),
            nn.ReLU(),
            nn.Linear(60, 60),
            nn.ReLU(),
            nn.Linear(60, 60),
            nn.ReLU(),
            nn.Linear(60, 10)
        )

        self.sequent.apply(self.__init_weights)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.sequent(x)
        return x
    
    def __init_weights(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.kaiming_normal_(m.weight)
    

In [162]:
torch.manual_seed(42)
model = Model()
model.to(device)

Model(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (sequent): Sequential(
    (0): Linear(in_features=784, out_features=60, bias=True)
    (1): ReLU()
    (2): Linear(in_features=60, out_features=60, bias=True)
    (3): ReLU()
    (4): Linear(in_features=60, out_features=60, bias=True)
    (5): ReLU()
    (6): Linear(in_features=60, out_features=10, bias=True)
  )
)

In [163]:
train_dataloader = torch.utils.data.DataLoader(list(zip(X_train, y_train)), batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(list(zip(X_test, y_test)), batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(list(zip(X_val, y_valid)), batch_size=32, shuffle=False)

In [164]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)

min_val_loss = float('inf')
early_stop = 0

for i in tqdm(range(30)):


    model.train()
    acc = 0
    total_loss = 0
    for brach, (X_batch, y_batch) in enumerate(train_dataloader):
        y_pred = model(X_batch)
        loss = F.cross_entropy(y_pred, y_batch)
        total_loss += loss
        acc += (torch.argmax(y_pred, dim=1) == y_batch).sum().item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    acc /= len(y_train)
    total_loss /= len(train_dataloader)
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for X_batch, y_batch in val_dataloader:
            y_pred = model(X_batch)
            val_loss += F.cross_entropy(y_pred, y_batch).item()
            val_acc += (torch.argmax(y_pred, dim=1) == y_batch).sum().item()
    
    val_loss /= len(val_dataloader)

    print(f"Epoch {i+1} - Accuracy: {acc:.4f} - Loss: {total_loss:.4f} - Val Loss: {val_loss:.4f}, Val accuracy: {val_acc/len(y_valid):.4f}")
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop = 0
    else:
        early_stop += 1
        if early_stop == 10:
            break

    scheduler.step(val_loss)
        



    

  3%|▎         | 1/30 [00:03<01:54,  3.94s/it]

Epoch 1 - Accuracy: 0.7926 - Loss: 0.5878 - Val Loss: 0.4813, Val accuracy: 0.8303


  7%|▋         | 2/30 [00:07<01:49,  3.90s/it]

Epoch 2 - Accuracy: 0.8329 - Loss: 0.4713 - Val Loss: 0.4447, Val accuracy: 0.8436


 10%|█         | 3/30 [00:11<01:48,  4.02s/it]

...

Epoch 28 - Accuracy: 0.8926 - Loss: 0.2880 - Val Loss: 0.3730, Val accuracy: 0.8665


 97%|█████████▋| 29/30 [02:08<00:04,  4.44s/it]

Epoch 29 - Accuracy: 0.8939 - Loss: 0.2867 - Val Loss: 0.3634, Val accuracy: 0.8726


100%|██████████| 30/30 [02:12<00:00,  4.43s/it]

Epoch 30 - Accuracy: 0.8931 - Loss: 0.2855 - Val Loss: 0.3919, Val accuracy: 0.8660
